In [1]:
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
from math import log

# Étape 1 : Crible segmenté pour lister les nombres premiers ≤ n
@jit(nopython=True)
def segmented_sieve(n):
    # Crible initial pour les petits nombres premiers jusqu'à sqrt(n)
    limit = int(np.sqrt(n)) + 1
    sieve = np.ones(limit + 1, dtype=np.bool_)
    sieve[0:2] = False
    for i in range(2, int(np.sqrt(limit)) + 1):
        if sieve[i]:
            sieve[i*i:limit+1:i] = False
    small_primes = np.where(sieve)[0]
    
    # Liste pour stocker tous les nombres premiers
    primes = []
    primes.extend(small_primes)
    
    # Crible segmenté
    segment_size = max(limit, 1000000)
    for low in range(limit, n + 1, segment_size):
        high = min(low + segment_size - 1, n)
        sieve = np.ones(high - low + 1, dtype=np.bool_)
        
        for p in small_primes:
            if p * p > high:
                break
            start = max(p * p, ((low + p - 1) // p) * p)
            sieve[start - low:high - low + 1:p] = False
        
        segment_primes = np.where(sieve)[0] + low
        primes.extend(segment_primes)
    
    return np.array(primes, dtype=np.int64)

# Étape 2 : Calcul des écarts gk et des ratios gk / log(pk)
def compute_gaps_and_ratios(primes):
    gaps = primes[1:] - primes[:-1]
    ratios = gaps / np.log(primes[:-1])
    return gaps, ratios

# Étape 3 : Histogramme des ratios et espérance empirique
def plot_histogram(ratios, title, filename):
    plt.figure(figsize=(10, 6))
    plt.hist(ratios, bins=100, density=True, alpha=0.7, color='blue', label='Ratios')
    plt.axvline(np.mean(ratios), color='red', linestyle='dashed', label=f'Espérance empirique = {np.mean(ratios):.3f}')
    plt.title(title)
    plt.xlabel('Ratio gk / log(pk)')
    plt.ylabel('Densité')
    plt.legend()
    plt.grid(True)
    plt.savefig(filename)
    plt.close()

# Étape 4 : Analyse pour n = 10^7 et n = 10^8
def analyze_prime_gaps(n, filename, title):
    print(f"Calcul des nombres premiers jusqu'à {n}...")
    primes = segmented_sieve(n)
    print(f"Nombre de premiers trouvés : {len(primes)}")
    
    gaps, ratios = compute_gaps_and_ratios(primes)
    mean_ratio = np.mean(ratios)
    print(f"Espérance empirique des ratios pour n={n} : {mean_ratio:.3f}")
    
    plot_histogram(ratios, title, filename)
    return ratios

# Étape 5 : Conjecture sur la loi limite
def compare_exponential(ratios, filename):
    plt.figure(figsize=(10, 6))
    plt.hist(ratios, bins=100, density=True, alpha=0.7, color='blue', label='Ratios')
    x = np.linspace(0, max(ratios), 100)
    plt.plot(x, np.exp(-x), 'r-', label='Densité exponentielle (λ=1)')
    plt.title('Comparaison avec une loi exponentielle')
    plt.xlabel('Ratio gk / log(pk)')
    plt.ylabel('Densité')
    plt.legend()
    plt.grid(True)
    plt.savefig(filename)
    plt.close()

# Exécution principale
if __name__ == "__main__":
    # Analyse pour n = 10^7
    ratios_1e7 = analyze_prime_gaps(int(1e7), 'histogram_1e7.png', 'Histogramme des ratios pour n=10^7')
    
    # Analyse pour n = 10^8 (si possible)
    ratios_1e8 = analyze_prime_gaps(int(1e8), 'histogram_1e8.png', 'Histogramme des ratios pour n=10^8')
    
    # Comparaison avec la loi exponentielle pour n = 10^8
    compare_exponential(ratios_1e8, 'exponential_comparison.png')
    
    print("Conjecture : Les ratios gk / log(pk) semblent suivre une loi exponentielle de paramètre λ=1.")

Calcul des nombres premiers jusqu'à 10000000...
Nombre de premiers trouvés : 664580
Espérance empirique des ratios pour n=10000000 : 1.001
Calcul des nombres premiers jusqu'à 100000000...
Nombre de premiers trouvés : 5761455
Espérance empirique des ratios pour n=100000000 : 1.000
Conjecture : Les ratios gk / log(pk) semblent suivre une loi exponentielle de paramètre λ=1.
